In [15]:
%pip install nltk
%pip install contractions
%pip install transformers
%pip install chardet
%pip install tqdm
%pip install --upgrade transformers accelerate
%pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
import re
import string
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaModel
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader, Dataset
# from transformers import DistilBertTokenizer, DistilBertModel, AdamW
from sklearn.preprocessing import LabelEncoder
import numpy as np
import chardet
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import requests
import json

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [17]:
data = pd.read_csv('/workspaces/misinformation/labeled_data_opinion_true.csv', encoding='utf-8')
data.shape

(2829, 10)

ClaimBuster is designed to detect factual claims in textual data. It provides a factual claim score (ranging from 0 to 1) that reflects how much a piece of text resembles a factual claim, with higher scores indicating a stronger factual claim.

In [18]:
api_key = "1d4a3eb3686441c0924adf0165146cb4"

def get_claimbuster_score(text):
    try:
        # Define the endpoint with the claim as part of it
        api_endpoint = f"https://idir.uta.edu/claimbuster/api/v2/score/text/{text}"
        request_headers = {"x-api-key": api_key}

        response = requests.get(url=api_endpoint, headers=request_headers)

        if response.status_code == 200:
            result = response.json()
            return result['results'][0]['score']  
        else:
            print(f"Error: {response.status_code}")
            return None  
    except Exception as e:
        print(f"Error occurred: {e}")
        return None 

data['claimbuster_score'] = data['text'].apply(get_claimbuster_score)

Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occurred: 0
Error occu

In [19]:
data.head()

,origin,Unnamed: 0,text,author.userName,misinformation,stance,isUSRelated,notes,opinion_label,true_label,claimbuster_score
0,T2,1459086,Unconfirmed reports claiming to originate from...,_HenryBolton,O,U,0,NaN,1,NaN,0.505682
1,T4,198836,Brexit is not the reason for the insecurity of...,_HenryBolton,O,P,0,NaN,1,NaN,0.495295
2,T2,1131342,I’ll be on @talkRADIO in around 5-10 (3:20pm i...,_HenryBolton,O,P,1,NaN,1,NaN,0.418709
3,T2,1345479,The judge hearing Djokovic’s deportation appea...,_HenryBolton,T,U,0,NaN,0,1.0,0.605284
4,T2,1116059,An order to a Border Force vessel to turn back...,_HenryBolton,O,U,0,NaN,1,NaN,0.530550


In [20]:
data['claimbuster_score'].describe()

count    2738.000000
mean        0.586638
std         0.170256
min         0.087925
25%         0.466782
50%         0.606765
75%         0.714399
max         0.920794
Name: claimbuster_score, dtype: float64

In [24]:
data['claimbuster_score'].isnull().sum()

np.int64(91)

In [27]:
data.to_csv('claimbuster.csv')

## Claim Spotting Models
https://github.com/utaresearch/claimbuster-spotter

### Transformer
https://colab.research.google.com/github/idirlab/claimspotter/blob/master/adv_transformer/adv_transformer-notebook.ipynb#scrollTo=V5DpLL03WJ-v

In [58]:
!git clone https://github.com/idirlab/claimspotter.git

Cloning into 'claimspotter'...
remote: Enumerating objects: 9856, done.
remote: Counting objects: 100% (706/706), done.
remote: Compressing objects: 100% (273/273), done.
remote: Total 9856 (delta 485), reused 592 (delta 433), pack-reused 9150 (from 1)
Receiving objects: 100% (9856/9856), 12.93 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (6904/6904), done.


In [86]:
%cd /workspaces/misinformation/claimspotter/adv_transformer/claimspotter/adv_transformer

/workspaces/misinformation/claimspotter/adv_transformer/claimspotter/adv_transformer


In [96]:
pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.


In [101]:
!pip install -r requirements.txt

  Using cached absl_py-1.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached aiofiles-0.5.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached astroid-2.4.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached asyncpg-0.20.1.tar.gz (734 kB)
  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.9.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached bert_tensorflow-1.0.1-py2.py3-none-any.whl.metadata (530 bytes)
  Using cached blis-0.4.1.tar.gz (1.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached boto-2.49.0-py2.py3-none-any.whl.metadata (7.3 kB)
  Using cached boto3-1.14.4-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached botocore-1.17.4-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached cachetools-4.1.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached certifi-2020.4.5.2-py2.py3-none-any.whl.metadata (2

In [75]:
import os

os.environ["BB"] = "./output/bb/"
os.environ["BBA"] = "./output/bba/"
os.environ["DB"] = "./output/db/"
os.environ["DBA"] = "./output/dba/"
os.environ["RB"] = "./output/rb/"
os.environ["RBA"] = "./output/rba/"

In [76]:
tweets = data['text']

In [83]:
%cd /workspaces/misinformation/claimspotter/adv_transformer/claimspotter

!python3 -m adv_transformer.train \
    --cs_model_dir=$BB \
    --cs_adv_train=False \
    --cs_gpu=0 \
    --cs_train_steps=5

/home/codespace/.local/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/workspaces/misinformation/claimspotter/adv_transformer/claimspotter
/workspaces/misinformation/claimspotter/adv_transformer/claimspotter/adv_transformer/core/utils/transformations.py:331: SyntaxWarning: invalid escape sequence '\/'
  sentence = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', 'url', sentence)
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/workspaces/misinformation/claimspotter/adv_transformer/claimspotter/adv_transformer/train.py", line 28, in <module>
    from adv_transformer.core.utils.data_loader import DataLoader
  File "/workspaces/misinformation/claimspotter/adv_transformer/claimspotter/adv_transformer/core/utils/data_loader.py", line 31, in <module>
    from . import transformations as transf
  File "/workspaces/misinformation/claimspotter/adv_transformer/claimspotter/adv_transformer/core/utils/transformations.py", line 24, in <module>
    imp

### Bi-Directional LSTM
https://colab.research.google.com/github/idirlab/claimspotter/blob/master/bidirectional_lstm/bilstm-notebook.ipynb#scrollTo=9s5qGyi5jWjU

In [35]:
text_data = data['text'].tolist()